# Import Data:

In [1]:
#list the data - the csv files in this directory are a result of the API code below
!ls Data

1998_housevote_meta.csv   2009_senatevote_pos.csv
1998_housevote_pos.csv	  2010_housevote_meta.csv
1998_senatevote_meta.csv  2010_housevote_pos.csv
1998_senatevote_pos.csv   2010_senatevote_meta.csv
1999_housevote_meta.csv   2010_senatevote_pos.csv
1999_housevote_pos.csv	  2011_housevote_meta.csv
1999_senatevote_meta.csv  2011_housevote_pos.csv
1999_senatevote_pos.csv   2011_senatevote_meta.csv
2000_housevote_meta.csv   2011_senatevote_pos.csv
2000_housevote_pos.csv	  2012_housevote_meta.csv
2000_senatevote_meta.csv  2012_housevote_pos.csv
2000_senatevote_pos.csv   2012_senatevote_meta.csv
2001_housevote_meta.csv   2012_senatevote_pos.csv
2001_housevote_pos.csv	  2013_housevote_meta.csv
2001_senatevote_meta.csv  2013_housevote_pos.csv
2001_senatevote_pos.csv   2013_senatevote_meta.csv
2002_housevote_meta.csv   2013_senatevote_pos.csv
2002_housevote_pos.csv	  2014_housevote_meta.csv
2002_senatevote_meta.csv  2014_housevote_pos.csv
2002_senatevote_pos.csv   2014_senatevote_meta.csv
2003_

In [2]:
!ls

Capstone Project Proposal - 2.pdf  Data Wrangling.ipynb    README.md
Data				   PROPUBLICA_API_KEY.TXT


### Get Pro Publica API working

In [3]:
import requests

#Load ProPublica API key without exposing it to github. Request one yourself. They're free.
#https://projects.propublica.org/api-docs/congress-api/
APIfile = open('./PROPUBLICA_API_KEY.TXT',)
APIkey = APIfile.read().replace('\n', '')

#simple unit tests - will throw an error if API key is malformed
assert type(APIkey) == str
assert len(APIkey) == 40

#create API key header
headers = {'X-API-Key': APIkey}

#API request function
def API_Request(end_point, verbose=True):
    '''for a given endpoint, return API response from Propublica api as a dictionary'''
    
    #send request using combined url, endpoint, and headers
    response = requests.get(url+end_point, headers=headers)
    
    #raise error if status code isn't 200. 
    response.raise_for_status()
    
    #return json response interpreted as a dictionary. 
    return response.json()

#load base url and test endpoint
url = 'https://api.propublica.org/congress/v1'
test_endpoint = '/members/new.json'

#test that the API is working. Prints 'OK' on succes
new_members = API_Request(test_endpoint)
assert new_members['status'] == 'OK'
assert new_members['results'][0]['num_results'] == '20'
print(new_members['status'])

OK


### OK, The API is working.  Let's import voting data

In [4]:
import pandas as pd
import numpy as np
from retrying import retry
from datetime import datetime

@retry(stop_max_attempt_number=5) #we're going to try this function 5 times until it returns a value
def API_attempts(endpoint):
    '''takes and endpoint and requests the API with it for a given number of times'''  
    #if we can't extract the status we'll keep trying, and if it's OK return the repsonse
    
    #initialize status as an error, this will change if we can extract it.
    status = 'Error fetching API response'
    
    #try to get the API response and extract status without throwing an error on failure
    try:
        #send endpoint to API function for request
        response = API_Request(endpoint)
        
        #extract the status of the API request
        status = response['status']   
    except:
        pass
    
    #if the status is not OK:
    if status != 'OK':
        #Say what the status is
        print(status, end=' ')
        
        #state that we're retrying
        print('retrying')
        
        raise
    else: 
        #return the response if status is ok
        return response
    
        
#let's create a function that will import vote positions for a given rollcall vote.
def import_vote(congress, chamber, session, roll_call):
    '''imports vote details and member positions of a given roll_call vote
        takes congress number 102-116 for House, 80-116 for Senate
        chamber(house|senate) session 1 for odd 2 for even-numbered years
        returns a dictionary of vote positions and API status'''
    #construct endpoint for API request
    for x in [congress, chamber, session, roll_call]:
        x = str(x)#                 
    call_endpoint = '/{}/{}/sessions/{}/votes/{}.json'.format(congress, chamber.lower(), session, roll_call)
    
    #let's attempt a few times to account for internet burps
    call_response = API_attempts(call_endpoint)
        
    #try to extract and return the vote positions
    try:
        vote = call_response['results']['votes']['vote']
        return vote
    except:
        return {}
    

def positions_df(metalist):
    '''return a dataframe of postions for a given list of vote metadata'''
    
    #initialize dataframe
    dfp = pd.DataFrame()
    
    #for every position in the list
    for congress, chamber, session, roll_call in metalist:
        
        #request the roll_call vote from the API
        vote = import_vote(congress, chamber, session, roll_call)
        
        #extract the positions from the rollcall vote data
        positions = vote.pop('positions')
        
        #turn the postions into a dataframe
        call_positions = pd.DataFrame(positions)
        
        #if both dataframes we are going to merge are not empty
        if not dfp.empty and not call_positions.empty:
            
            #handle duplicated positions for malformed returns
            dups = call_positions.duplicated()
            dups = dups.sum()
            if dups > 0: 
                call_postions = call_postions.drop_duplicates()
                print('dropped{}duplicates'.format(dups))
        
            #MERGE THE POSITIONS FOR A GIVEN ROLL CALL VOTE INTO DATAFRAME <- core of function 
            dfp = pd.merge(dfp, call_positions, on=mergelst, how='outer')
            
            #let us know it's working
            print('.', end='')
            
        #If dfp is empty this is the first run and we need to initialize it
        elif dfp.empty: 
            #make it equal to call_positions
            dfp = call_positions
            
            #make a list of the columns that will be used to merge the call positions
            mergelst = dfp.columns.tolist()[:-1]
            
            #move party and state to front of merglist
            for x in ['state','party']:
                mergelst.insert(0, mergelst.pop(mergelst.index(x)))
            
            #alert that we've started
            print('\ndf created on', congress, chamber, session, roll_call, end=' ')
        
        #handle empty postitions dataframe:
        elif call_positions.empty:
            
            #let us know there was a problem
            print('\nno vote postions available for', congress, chamber, session, roll_call, end=' ')
            
            #append a series of Nan values
            ht = dfp.shape[0]
            nans = pd.Series([np.nan for _ in range(ht)])
            print ('dfp.shape:', dfp.shape)
            print ('Appending {} NaNs'.format(len(nans)))
            dfp = pd.concat([dfp, nans], axis = 1)
            print(dfp.iloc[:3,-4:])
        
        #handle unknown issue
        else:
            print('\nunknown error for', congress, chamber, session, roll_call, end=' ')
    
    #create a multindex of partymembers
    dfp = dfp.set_index(mergelst).transpose()
    
    #after transposing use the metalist as the index
    dfp.index = metalist
    
    #return the complete data frame
    return dfp 

def import_month(year, chamber, month):
    '''import vote metadata for a given year or month and return a dataframe '''
    
    #Stop everthing if we are using a year before 1989 or after now
    assert 1989 <= year <= int(datetime.now().year), "year out of range"
    
    #construct endpoint
    call_endpoint = '/{}/votes/{}/{}.json'.format(chamber.lower(), str(year), str(month))
    
    #use endpoint to request the API robustly with attempts function
    response = API_attempts(call_endpoint)
    
    #Extract the votes from the response
    votes = response['results']['votes']

    #create a dataframe out of the votes
    votesdf = pd.DataFrame(votes)
    
    if not votesdf.empty:
        votesdf = votesdf.set_index(['congress','chamber','session','roll_call']).sort_index()
    
    #return the months dataframe
    return votesdf

def import_year(year, chamber):
    '''import votes based on dates, return a dataframe of appended votes'''
    
    
    #So we know we started:
    print('importing', chamber, year, end=' ')
    #loop through the months 
    for month in range(1,13):
        
        #import the first month
        if month == 1:
            yeardf = import_month(year, chamber, month)
       
        #import the month of votes as a df and append it to the list 
        else:
            df = import_month(year, chamber, month)
            if not df.empty:
                yeardf = yeardf.append(df)
            else:
                print('skip', end=' ')
        #print something so we know it's working
        print(month, end=' ')
        
    #return the dataframe
    return yeardf

def save_years(start_y, end_y):
    '''take a range of years and save a series of dataframes for those years in both the hosue and senate'''
    #iterate through the range of years
    for year in range(start_y, end_y+1):
        #alternate between senate and house
        for chamber in ['senate','house']:
            #import vote metadata dataframe
            votemetadf = import_year(year, chamber)
            
            #import vote positions for given dataframe
            voteposdf = positions_df(votemetadf.index)
            
            #construct paths
            #pathhead = 'Data/'+str(year)+'_'+chamber+'vote_'
            pathhead = 'Data/{}_{}vote_'.format(str(year), chamber)
            metapath = pathhead + 'meta.csv'
            pospath = pathhead + 'pos.csv'
            
            #announce what we're doing
            #print('\nsaving '+str(year)+' '+chamber+' as '+metapath+' and '+pospath)
            print('\nsaving {} {} as {} and {}'.format(str(year), chamber, metapath, pospath))
        
            #save data frames as csv in data/csv directory
            votemetadf.to_csv(metapath)
            voteposdf.to_csv(pospath)
            
            #save data frames as pickle in data/pickle directory?
            
            
#tests - comment out when working

#senate_month_test = import_month(2019, 'senate', 1)
#senate_month_test

#senate_2019_test = import_year(2019, 'senate')
#senate_2019_test

#positions = positions_df(senate_2019_test.index)
#positions

#save_years(2019, 2019)

In [11]:
save_years(1998,1999)

importing senate 1998 1 2 3 4 5 6 7 skip 8 9 10 skip 11 skip 12 
df created on 105 Senate 2 1 .........................................................................................................................................................................................................................................................................................................................
saving 1998 senate as Data/1998_senatevote_meta.csv and Data/1998_senatevote_pos.csv
importing house 1998 1 2 3 4 5 6 7 8 9 10 skip 11 12 
df created on 105 House 2 1 ........................Error fetching API response retrying
............................................................................................................................................................................................................................................................................................................................................................................

In [43]:
save_years(2009,2009)

importing senate 2009 1 2 3 4 5 6 7 8 9 10 11 12 
df created on 111 Senate 1 1 ............................................................................................................................................................................................................................................................................................................................................................................................................
saving 2009 senate as Data/2009_senatevote_meta.csv and Data/2009_senatevote_pos.csv
importing house 2009 1 2 3 4 5 6 7 skip 8 9 10 11 12 
df created on 111 House 1 1 .....................................................................................................................................................................................................................................................................................................................................................................

In [5]:
save_years(2014,2014)

importing senate 2014 1 2 3 4 5 6 7 skip 8 9 skip 10 11 12 
df created on 113 Senate 2 1 .............................................................................................................................................................................................................................................................................................................................................................................
saving 2014 senate as Data/2014_senatevote_meta.csv and Data/2014_senatevote_pos.csv
importing house 2014 1 2 3 4 5 6 7 8 9 skip 10 11 12 
df created on 113 House 2 1 
no vote postions available for 113 House 2 2 dfp.shape: (432, 7)
Appending 432
  party state vote_position   0
0     R    AL       Present NaN
1     R    MI       Present NaN
2     R    NV    Not Voting NaN
..........................................................................................................................................................................

In [7]:
test_dfsenate = pd.read_csv('Data/2019_senatevote_pos.csv', header = [0,1,2,3,4], index_col = [0,1,2,3])
test_dfsenate.head()

party                                            R             D  \
state                                           TN            WI   
dw_nominate                                  0.323        -0.521   
member_id                                  A000360       B001230   
name                               Lamar Alexander Tammy Baldwin   
congress chamber session roll_call                                 
116      Senate  1       1                     Yes            No   
                         2                     Yes            No   
                         3                     Yes            No   
                         4                     Yes            No   
                         5                      No           Yes   

party                                          R              D  \
state                                         WY             CO   
dw_nominate                                0.534         -0.208   
member_id                                B001261        B001267   
name                               John Barrasso Michael Bennet   
congress chamber session roll_call                                
116      Senate  1       1                   Yes             No   
                         2                   Yes             No   
                         3                   Yes             No   
                         4                   Yes             No   
                         5                    No            Yes   

party                                             R                  D  \
state                                            TN                 CT   
dw_nominate                                   0.614             -0.411   
member_id                                   B001243            B001277   
name                               Marsha Blackburn Richard Blumenthal   
congress chamber session roll_call                                       
116      Senate  1       1                      Yes                 No   
                         2                      Yes                 No   
                         3                      Yes                 No   
                         4                      Yes                 No   
                         5                       No                Yes   

party                                      R           D            R  \
state                                     MO          NJ           AR   
dw_nominate                            0.429      -0.618        0.401   
member_id                            B000575     B001288      B001236   
name                               Roy Blunt Cory Booker John Boozman   
congress chamber session roll_call                                      
116      Senate  1       1               Yes          No          Yes   
                         2               Yes          No          Yes   
                         3               Yes          No          Yes   
                         4               Yes          No           No   
                         5                No         Yes          Yes   

party                                            ...                  \
state                                      IN    ...              NC   
dw_nominate                               nan    ...           0.419   
member_id                             B001310    ...         T000476   
name                               Mike Braun    ...     Thom Tillis   
congress chamber session roll_call               ...                   
116      Senate  1       1                Yes    ...             Yes   
                         2                Yes    ...             Yes   
                         3                Yes    ...             Yes   
                         4                Yes    ...             Yes   
                         5                 No    ...              No   

party                                                        D  \
state                              

In [8]:
test_dfhouse = pd.read_csv('Data/2019_housevote_pos.csv', header = [0,1,2,3,4,5], index_col = [0,1,2,3])
test_dfhouse.head()

party                                          R          D  \
state                                         LA         NC   
district                                       5         12   
dw_nominate                                0.514     -0.469   
member_id                                A000374    A000370   
name                               Ralph Abraham Alma Adams   
congress chamber session roll_call                            
116      House   1       1               Present    Present   
                         2              McCarthy     Pelosi   
                         3                    No        Yes   
                         4                    No        Yes   
                         5                   Yes         No   

party                                               R            D          R  \
state                                              AL           CA         GA   
district                                            4           31         12   
dw_nominate                                     0.363       -0.287      0.634   
member_id                                     A000055      A000371    A000372   
name                               Robert B. Aderholt Pete Aguilar Rick Allen   
congress chamber session roll_call                                              
116      House   1       1                    Present      Present    Present   
                         2                   McCarthy       Pelosi   McCarthy   
                         3                         No          Yes         No   
                         4                         No          Yes         No   
                         5                        Yes           No        Yes   

party                                         D            R              \
state                                        TX           MI          NV   
district                                     32            3           2   
dw_nominate                                 nan        0.641        0.38   
member_id                               A000376      A000367     A000369   
name                               Colin Allred Justin Amash Mark Amodei   
congress chamber session roll_call                                         
116      House   1       1              Present      Present     Present   
                         2               Pelosi       Massie    McCarthy   
                         3                  Yes           No          No   
                         4                  Yes           No          No   
                         5                   No          Yes         Yes   

party                                                               \
state                                           ND              TX   
district                                         1              19   
dw_nominate                                    nan           0.541   
member_id                                  A000377         A000375   
name                               Kelly Armstrong Jodey Arrington   
congress chamber session roll_call                                   
116      House   1       1                 Present         Present   
                         2                McCarthy        McCarthy   
                         3                      No              No   
                         4                      No              No   
                         5                     Yes             Yes   

party                                      ...                    D         R  \
state                                      ...                   KY        FL   
district                                   ...                    3         3   
dw_nominate                                ...               -0.378     0.701   
member_id                                  ...              Y000062   Y000065   
name                                       ...         John Yarmuth  Ted Yoho   
congress chamber session roll_

In [9]:
testmetahouse = pd.read_csv('Data/2019_housevote_meta.csv', index_col = [0,1,2,3])
testmetahouse.head()

bill  \
congress chamber session roll_call                                                      
116      House   1       1          {'bill_id': 'quorum-116', 'number': 'QUORUM', ...   
                         2                                                         {}   
                         3          {'bill_id': 'hres5-116', 'number': 'H.RES.5', ...   
                         4          {'bill_id': 'hres5-116', 'number': 'H.RES.5', ...   
                         5          {'bill_id': 'hres5-116', 'number': 'H.RES.5', ...   

                                          date  \
congress chamber session roll_call               
116      House   1       1          2019-01-03   
                         2          2019-01-03   
                         3          2019-01-03   
                         4          2019-01-03   
                         5          2019-01-03   

                                                                           democratic  \
congress chamber session roll_call                                                      
116      House   1       1          {'yes': 0, 'no': 0, 'present': 234, 'not_votin...   
                         2          {'Pelosi': 220, 'Biden': 1, 'Present': 3, 'Duc...   
                         3          {'yes': 230, 'no': 0, 'present': 0, 'not_votin...   
                         4          {'yes': 233, 'no': 0, 'present': 0, 'not_votin...   
                         5          {'yes': 0, 'no': 232, 'present': 0, 'not_votin...   

                                                                          description  \
congress chamber session roll_call                                                      
116      House   1       1                                                        NaN   
                         2                                                        NaN   
                         3          Providing for consideration of H.Res. 6, adopt...   
                         4          Providing for consideration of H.Res. 6, adopt...   
                         5          Providing for consideration of H.Res. 6, adopt...   

                                                                          independent  \
congress chamber session roll_call                                                      
116      House   1       1          {'yes': 0, 'no': 0, 'present': 0, 'not_voting'...   
                         2                                                         {}   
                         3          {'yes': 0, 'no': 0, 'present': 0, 'not_voting'...   
                         4          {'yes': 0, 'no': 0, 'present': 0, 'not_voting'...   
                         5          {'yes': 0, 'no': 0, 'present': 0, 'not_voting'...   

                                                                  question  \
congress chamber session roll_call                                           
116      House   1       1                                  Call by States   
                         2                         Election of the Speaker   
                         3          On Motion to Table the Motion to Refer   
                         4               On Ordering the Previous Question   
                         5           On Motion to Commit with Instructions   

                                    question_text  \
congress chamber session roll_call                  
116      House   1       1                    NaN   
                         2                    NaN   
                         3                    NaN   
                         4                    NaN   
                         5                    NaN   

                                                                           republican  \
congress chamber session roll_call                                                      
116      House   1       1          {'yes': 0, 'no': 0, 'present': 197, 'not_votin...   
                         2          {'McCarthy':

In [10]:
testmetasenate = pd.read_csv('Data/2019_senatevote_meta.csv', index_col = [0,1,2,3])
testmetasenate.head()

bill  \
congress chamber session roll_call                                                      
116      Senate  1       1          {'bill_id': 's1-116', 'number': 'S.1', 'sponso...   
                         2          {'bill_id': 's1-116', 'number': 'S.1', 'sponso...   
                         3          {'bill_id': 's1-116', 'number': 'S.1', 'sponso...   
                         4          {'bill_id': 'sjres2-116', 'number': 'S.J.RES.2...   
                         5          {'bill_id': 'sjres2-116', 'number': 'S.J.RES.2...   

                                          date  \
congress chamber session roll_call               
116      Senate  1       1          2019-01-08   
                         2          2019-01-10   
                         3          2019-01-14   
                         4          2019-01-15   
                         5          2019-01-15   

                                                                           democratic  \
congress chamber session roll_call                                                      
116      Senate  1       1          {'yes': 4, 'no': 41, 'present': 0, 'not_voting...   
                         2          {'yes': 4, 'no': 41, 'present': 0, 'not_voting...   
                         3          {'yes': 3, 'no': 40, 'present': 0, 'not_voting...   
                         4          {'yes': 0, 'no': 44, 'present': 0, 'not_voting...   
                         5          {'yes': 44, 'no': 0, 'present': 0, 'not_voting...   

                                                                          description  \
congress chamber session roll_call                                                      
116      Senate  1       1          A bill to make improvements to certain defense...   
                         2          A bill to make improvements to certain defense...   
                         3          A bill to make improvements to certain defense...   
                         4          A joint resolution disapproving the President'...   
                         5          A joint resolution disapproving the President'...   

                                    document_number  \
congress chamber session roll_call                    
116      Senate  1       1                      1.0   
                         2                      1.0   
                         3                      1.0   
                         4                      2.0   
                         5                      2.0   

                                                                       document_title  \
congress chamber session roll_call                                                      
116      Senate  1       1          A bill to make improvements to certain defense...   
                         2          A bill to make improvements to certain defense...   
                         3          A bill to make improvements to certain defense...   
                         4          A joint resolution disapproving the President&...   
                         5          A joint resolution disapproving the President&...   

                                                                          independent  \
congress chamber session roll_call                                                      
116      Senate  1       1          {'yes': 0, 'no': 2, 'present': 0, 'not_voting'...   
                         2          {'yes': 0, 'no': 2, 'present': 0, 'not_voting'...   
                         3          {'yes': 0, 'no': 2, 'present': 0, 'not_voting'...   
                         4          {'yes': 0, 'no': 2, 'present': 0, 'not_voting'...   
                         5          {'yes': 2, 'no': 0, 'present': 0, 'not_voting'...   

                                                               question  \
congress chamber session roll_call                                        
116      Senate  1       1          On Cloture on the Motion to Proceed   